In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add the parent directory to the Python path
sys.path.append('..')

from src.data import load_data
from src.models import create_ga_model
from src.genetic_algorithm import genetic_algorithm
from src.utils import set_seeds, create_results_directory
from src.visualization import plot_predictions, plot_fitness_history, plot_time_series_predictions

# Set random seeds for reproducibility
set_seeds(42)

# Create a results directory
results_dir = create_results_directory()
print(f"Results will be saved in: {results_dir}")

2024-10-21 08:18:31.649450: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-21 08:18:31.656456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 08:18:31.664481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 08:18:31.666682: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 08:18:31.673007: I tensorflow/core/platform/cpu_feature_guar

Results will be saved in: /home/dkat/computational-intelligence/src/../results/20241021_081832


In [2]:
# Load the data using TimeseriesGenerator
train_generator, val_generator, test_generator, scaler_X, scaler_y = load_data()

# Print data sequences
print("Data sequences:")
print(f"Train sequences: {len(train_generator)}")
print(f"Validation sequences: {len(val_generator)}")
print(f"Test sequences: {len(test_generator)}")

# Get input shape from one batch
x_batch, y_batch = train_generator[0]
input_shape = x_batch.shape[1:]  # Exclude batch size dimension
print(f"Input shape: {input_shape}")
print(f"Output shape: {y_batch.shape}")

2024-10-21 08:18:34,203 - INFO - Loading data from ../data/raw/Train.csv
2024-10-21 08:18:34,275 - INFO - Train data shape: (140160, 12)
2024-10-21 08:18:34,278 - INFO - Input columns: ['Temp_2m', 'RelHum_2m', 'DP_2m', 'WS_10m', 'WS_100m', 'WD_10m', 'WD_100m', 'WG_10m']
2024-10-21 08:18:34,278 - INFO - Output column: Power
2024-10-21 08:18:34,278 - INFO - Training set shape: (98112, 9)
2024-10-21 08:18:34,279 - INFO - Validation set shape: (21024, 9)
2024-10-21 08:18:34,279 - INFO - Test set shape: (21024, 9)
2024-10-21 08:18:34,286 - INFO - Data scaling and sequence generation completed


Data sequences:
Train sequences: 3066
Validation sequences: 657
Test sequences: 657
Input shape: (12, 8)
Output shape: (32, 1)


In [3]:
# Define model architecture
layer_sizes = [256, 128, 64]

# Create the model
model = create_ga_model(input_shape, layer_sizes)
model.summary()

/home/dkat/computational-intelligence/venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1729498716.661525  267557 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729498716.675005  267557 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729498716.675037  267557 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729498716.676545  267557 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,049 (258.00 KB)

 Trainable params: 66,049 (258.00 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Set GA parameters
ga_params = {
    "population_size": 10,
    "generations": 100,
    "tournament_size": 3,
    "mutation_rate": 0.01,
    "mutation_scale": 0.15,
    "patience": 10  # Early stopping patience
}

# Run Genetic Algorithm
best_model, fitness_history, eval_results = genetic_algorithm(
    model,
    train_generator,
    val_generator,
    test_generator,
    scaler_y,
    results_dir=results_dir,
    **ga_params
)

Generation 0


I0000 00:00:1729498720.387090  267696 service.cc:146] XLA service 0x7f3640016090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729498720.387115  267696 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-10-21 08:18:40.390459: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-21 08:18:40.406988: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1729498721.028408  267696 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Best fitness: 1.432694
Generation 1
Best fitness: 1.367672
Generation 2
Best fitness: 1.265302
Generation 3
Best fitness: 1.268553
Generation 4
Best fitness: 1.259705
Generation 5
Best fitness: 1.297379
Generation 6
Best fitness: 1.391866
Generation 7
Best fitness: 1.442668
Generation 8
Best fitness: 1.455039
Generation 9
Best fitness: 1.437023
Generation 10
Best fitness: 1.545541
Generation 11
Best fitness: 1.485692
Generation 12
Best fitness: 1.523307
Generation 13
Best fitness: 1.521673
Generation 14
Best fitness: 1.611616
Generation 15
Best fitness: 1.532432
Generation 16
Best fitness: 1.583427
Generation 17
Best fitness: 1.696445
Generation 18
Best fitness: 1.711934
Generation 19
Best fitness: 1.706090
Generation 20
Best fitness: 1.672283
Generation 21
Best fitness: 1.745882
Generation 22
Best fitness: 1.770400
Generation 23
Best fitness: 1.761394
Generation 24
Best fitness: 1.778328
Generation 25
Best fitness: 1.747232
Generation 26
Best fitness: 1.773901
Generation 27
Best fitne

2024-10-21 08:21:21,766 - INFO - Model weights saved to /home/dkat/computational-intelligence/src/../results/20241021_081832/best_model.weights.h5
2024-10-21 08:21:21,766 - INFO - Evaluation metrics saved to /home/dkat/computational-intelligence/src/../results/20241021_081832/evaluation_metrics.txt


Test Loss (MSE): 0.396326
Test RMSE: 0.1628
Test R2 Score: 0.5075


In [5]:
# After training
print("\nEvaluation Results:")
print(f"Test MSE: {eval_results['mse']:.4f}")
print(f"Test RMSE: {eval_results['rmse']:.4f}")
print(f"Test R2 Score: {eval_results['r2']:.4f}")

# Plot fitness history
plot_fitness_history(fitness_history, save_path=os.path.join(results_dir, 'ga_fitness_history.png'))

# Plot predictions
plot_predictions(
    eval_results['actual'],
    eval_results['predictions'],
    "GA Model Predictions",
    save_path=os.path.join(results_dir, 'ga_predictions.png')
)

# Plot time series predictions
plot_time_series_predictions(
    eval_results['actual'],
    eval_results['predictions'],
    title="GA Model Time Series Predictions",
    save_path=os.path.join(results_dir, 'ga_time_series_predictions.png')
)

# Display the plots in the notebook
plt.show()

2024-10-21 08:21:33,868 - WARNING - Seaborn style not found. Using default style.
2024-10-21 08:21:33,870 - INFO - Plot style set
2024-10-21 08:21:33,941 - INFO - Fitness history plot saved to /home/dkat/computational-intelligence/src/../results/20241021_081832/ga_fitness_history.png
2024-10-21 08:21:33,941 - WARNING - Seaborn style not found. Using default style.
2024-10-21 08:21:33,942 - INFO - Plot style set



Evaluation Results:
Test MSE: 0.3963
Test RMSE: 0.1628
Test R2 Score: 0.5075


2024-10-21 08:21:34,442 - INFO - Predictions plot saved to /home/dkat/computational-intelligence/src/../results/20241021_081832/ga_predictions.png
2024-10-21 08:21:34,442 - WARNING - Seaborn style not found. Using default style.
2024-10-21 08:21:34,443 - INFO - Plot style set
2024-10-21 08:21:34,628 - INFO - Time series predictions plot saved to /home/dkat/computational-intelligence/src/../results/20241021_081832/ga_time_series_predictions.png
